In [ ]:
#IMPORTS

import numpy as np 
import pandas
import math

import matplotlib.pyplot as plt
import os
import scipy
from scipy import signal
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Resizing
import random

!pip install -q nnAudio
import nnAudio
from nnAudio.Spectrogram import CQT1992v2
import torch

from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns

import librosa
import librosa.display



In [ ]:
#PREPROCESSING

#Dictionary of data ID -> labels
labels_df = pandas.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
dic = labels_df.set_index('id').to_dict()
labels = dic['target']
print("mean label: ",np.mean(np.array(list(labels.values()))))
print(len(labels.values()), "total training data")

#Split list of data ID into train & validation
id_data = list(labels.keys())
data_count = len(id_data)
train_count = math.floor(data_count * .95)
val_count = math.ceil(data_count * .05)
assert(val_count + train_count == data_count)
id_train = id_data[0:train_count]
id_val   = id_data[train_count:data_count] 
random.shuffle(id_train)
random.shuffle(id_val)

In [ ]:
#DATA TRANSFORMATION METHODS

window = signal.windows.tukey(4096)

def spec(waves,transform=CQT1992v2(sr=2048, hop_length=64, fmin=20, fmax=1024)): 
    #Windows sequential data -> normalizes -> converts to 3D freq. domain -> transpose
    
    waves = np.multiply(window,waves)
    waves = np.hstack(waves)
    waves = waves / np.max(waves)
    waves = torch.from_numpy(waves).float()
    image = transform(waves)
    image = np.array(image)
    image = np.transpose(image,(1,2,0))
    return image

def id2path(idx,is_train=True):
    #Takes data ID and returns file path locating numpy array
    
    path = '../input/g2net-gravitational-wave-detection'
    if is_train:
        path += '/train/'+idx[0]+'/'+idx[1]+'/'+idx[2]+'/'+idx+'.npy'
    else:
        path += '/test/'+idx[0]+'/'+idx[1]+'/'+idx[2]+'/'+idx+'.npy'
    return path

def normalize(x, triple = True):
    #Scale matrix of 3 vectors so each vector is in [-1,1]
    #Currently not used, currently scaling all 3 by max of all 3
    
    if triple:
        x0 = x[0] * 1/np.max(np.abs(x[0]))
        x1 = x[1] * 1/np.max(np.abs(x[1]))
        x2 = x[2] * 1/np.max(np.abs(x[2]))
        return np.array([x0,x1,x2])
    else:
        return x * 1/np.max(np.abs(x))


sos = signal.butter(8, [20,1000], 'bandpass', fs=2048, output='sos')

def bandpass(x):
    #Windows sequential data -> filters out frequencies not in [20,1000]
    
    x[0] = np.multiply(window,x[0])
    x[1] = np.multiply(window,x[1])
    x[2] = np.multiply(window,x[2])
    x[0] = signal.sosfilt(sos, x[0])
    x[1] = signal.sosfilt(sos, x[1])
    x[2] = signal.sosfilt(sos, x[2])
    return x

In [ ]:
#GENERATORS

#Generates 3D image data (spectrograms, frequency x intensity x time)
def generate_data(id_list=None, batch_size=32):
    inputs      = []
    targets     = []
    batch_count = 0
    
    
    random.shuffle(id_list)
    
    while True:
        for idx in id_list:
            inputs.append(spec(np.load(id2path(idx))))
            targets.append(labels[idx])
            batch_count += 1
            if batch_count >= batch_size: 
                yield (np.array(inputs), np.array(targets))
                inputs = []
                targets = []
                contexts = []
                batch_count = 0
             
            

#Generates 2D wave data (amplitude x time)
def generate_seq_data(id_list=None, batch_size=32):
    inputs      = []
    targets     = []
    batch_count = 0
    
    random.shuffle(id_list)
    
    while True:
        for idx in id_list:
            seq = np.transpose(bandpass(np.load(id2path(idx))))
            inputs.append(seq/np.max(seq))
            targets.append(labels[idx])
            batch_count += 1
            if batch_count >= batch_size: 
                yield (np.array(inputs), np.array(targets))
                inputs = []
                targets = []
                contexts = []
                batch_count = 0

In [ ]:
#Custom 1D ConvNet (Takes 1D Sequential data)

inputs1 = layers.Input(shape=(4096,3))
scale = 4
#Large Filter
large_conv1 = layers.Conv1D(filters=16*scale, kernel_size = 64, strides=5,  activation='relu')(inputs1)
large_conv2 = layers.Conv1D(filters=32*scale, kernel_size = 32, strides=4,  activation='relu', dilation_rate=1)(large_conv1)
large_conv3 = layers.Conv1D(filters=48*scale, kernel_size = 16, strides=1,  activation='relu', dilation_rate=2)(large_conv2)
large_pool = layers.MaxPooling1D()(large_conv3)

#Medium Filter
med_conv1 = layers.Conv1D(filters=16*scale, kernel_size = 16, strides=4, activation='relu')(inputs1)
med_conv2 = layers.Conv1D(filters=32*scale, kernel_size = 8, strides=2,  activation='relu', dilation_rate=1)(med_conv1)
med_conv3 = layers.Conv1D(filters=48*scale, kernel_size = 4, strides=1,  activation='relu', dilation_rate=2)(med_conv2)
med_pool = layers.MaxPooling1D()(med_conv3)

#Small Filter
small_conv1 = layers.Conv1D(filters=16*scale, kernel_size = 6, strides=2,  activation='relu')(inputs1)
small_conv2 = layers.Conv1D(filters=32*scale, kernel_size = 4, strides=1,  activation='relu', dilation_rate=1)(small_conv1)
small_conv3 = layers.Conv1D(filters=48*scale, kernel_size = 2, strides=1,  activation='relu', dilation_rate=2)(small_conv2)
small_pool = layers.MaxPooling1D()(small_conv3)

#Concatenate Filters
concat = layers.Concatenate(axis=1)([large_pool,med_pool,small_pool])
reshape = layers.Reshape((1352,48*scale,1))(concat)
conv2d = layers.Conv2D(filters=64, kernel_size=(16,16),strides=(8,8), activation='relu')(reshape)
pool = layers.GlobalMaxPool2D()(conv2d)
flat = layers.Flatten()(pool)
dense = layers.Dense(32, activation='relu')(flat)
drop = layers.Dropout(.5)(dense)
output = layers.Dense(1, activation='sigmoid')(drop)
model1 = keras.Model(inputs1,output)

model1.summary()
opt = tf.keras.optimizers.Adam(
    learning_rate=1e-4,
)

model1.compile(loss='binary_crossentropy', 
              optimizer=opt, 
              metrics=['accuracy','AUC'])

In [ ]:
#Pretrained 2D ConvNet (Takes 3D image data)

!pip install -U efficientnet
import efficientnet.keras as efn

inputs = layers.Input(shape=(69,193,1))
reshaped = layers.Conv2D(3,(3,3),padding='valid', activation='relu')(inputs) 
#preproc = preprocess_input(reshaped)
inception = efn.EfficientNetB0(include_top=False,input_shape=())(reshaped)
flat = layers.GlobalMaxPool2D()(inception)
drop = layers.Dropout(.5)(flat)
#dense = layers.Dense(32, activation='relu')(drop)
outputs = layers.Dense(1, activation='sigmoid')(drop)
model2 = keras.Model(inputs, outputs)

model2.summary()

opt = tf.keras.optimizers.Adam(
    learning_rate=.0001,
)

model2.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy','AUC'])

In [ ]:
#Shuffle and instantiate data generators
#In this case 1D

random.shuffle(id_train)
batch_size = 256
generator  = generate_seq_data(id_list = id_train,batch_size=batch_size)
generator_val  = generate_seq_data(id_list = id_val,batch_size=batch_size)
train_steps_per_epoch = train_count / batch_size
val_steps   = 64#val_count / batch_size

In [ ]:
#Train

model2.fit(generator,
          validation_data=generator_val,
          epochs=3,
          steps_per_epoch = train_steps_per_epoch,
          validation_steps = val_steps)

#Current results are .75 AUC for model1 and .85 AUC for model2
#Next step is to merge models into an ensemble, possibly train together. 
#Issue is 1d vs 2d conv require different learning rates. Pretrain then model3 = keras.Model([model1,model2],output)?
#Can also just merge their prediction vectors